I wanted to check what the upstream and downstream regions of the phastest predicted sites are to answer if: 
- phages that are similar have specific region they like to integarate
- are there any common features or genes that are present in this regions

I will be doing this using the fasta files present in `/Users/pjoglekar/work/pseudomonas/Pseudomonas_data_from_selected_genomes/pacbio_genomes/fasta_files`. I extracted the regions.txt from each Phastest predicted region and created a merged summary for these files in the tsv format available here `/Users/pjoglekar/work/pseudomonas/Pseudomonas_data_from_selected_genomes/pacbio_genomes/statistical_analysis/phastest_merge_summaries.tsv`

In [2]:
#Step 1: import required packages

import pandas as pd
from Bio import SeqIO
import os

# Create output directories if they don't exist
os.makedirs("upstream_seq_1500", exist_ok=True)
os.makedirs("downstream_seq_1500", exist_ok=True)

In [3]:
#Step 2: set input files and process the file names

#paths for files
# Paths
tsv_file = '/Users/pjoglekar/work/pseudomonas/Pseudomonas_data_from_selected_genomes/pacbio_genomes/statistical_analysis/phastest_merge_summaries.tsv'
fasta_dir = '/Users/pjoglekar/work/pseudomonas/Pseudomonas_data_from_selected_genomes/pacbio_genomes/fasta_files'


In [4]:

# Read TSV
df = pd.read_csv(tsv_file, sep='\t')

#Step 3:  Prepare output table
output_rows = []

for idx, row in df.iterrows():
    organism = row['ORGANISM']
    region = row['REGION']
    region_length = row['REGION_LENGTH']
    completeness = row['COMPLETENESS(score)']
    region_position = row['REGION_POSITION']

    fasta_base = organism.split('_contig')[0]
    contig_name = organism
    fasta_file = f"{fasta_dir}/{fasta_base}.fasta"

    for record in SeqIO.parse(fasta_file, "fasta"):
        if record.id == contig_name:
            seq = record.seq
            if '-' in region_position:
                start, end = map(int, region_position.split('-'))
            else:
                start = end = int(region_position)

            up_start = max(0, start-1500)
            up_end = start
            down_start = end
            down_end = min(len(seq), end+1500)

            # Track which extractions are valid
            valid_upstream = up_end > up_start and up_end <= len(seq)
            valid_downstream = down_end > down_start and down_end <= len(seq)

            # Extract and write only valid sequences
            if valid_upstream:
                upstream_seq = seq[up_start:up_end]
                up_fasta = f"upstream_seq_1500/{fasta_base}_{region}_upstream.fasta"
                SeqIO.write([record[up_start:up_end]], up_fasta, "fasta")
            else:
                print(f"Skipping upstream for region {region} in {organism}: invalid coordinates ({up_start}-{up_end})")

            if valid_downstream:
                downstream_seq = seq[down_start:down_end]
                down_fasta = f"downstream_seq_1500/{fasta_base}_{region}_downstream.fasta"
                SeqIO.write([record[down_start:down_end]], down_fasta, "fasta")
            else:
                print(f"Skipping downstream for region {region} in {organism}: invalid coordinates ({down_start}-{down_end})")

            # Record output, showing which coordinates were valid
            output_rows.append({
                'ORGANISM': organism,
                'REGION': region,
                'REGION_POSITION': region_position,
                'UPSTREAM_COORDS': f"{up_start}-{up_end}" if valid_upstream else "invalid",
                'DOWNSTREAM_COORDS': f"{down_start}-{down_end}" if valid_downstream else "invalid"
            })
            break
        


Skipping upstream for region 1 in Ps_papulans_renamed_contig_1: invalid coordinates (234205-235705)
Skipping downstream for region 1 in Ps_papulans_renamed_contig_1: invalid coordinates (275183-103811)
Skipping upstream for region 2 in Ps_papulans_renamed_contig_1: invalid coordinates (354679-356179)
Skipping downstream for region 2 in Ps_papulans_renamed_contig_1: invalid coordinates (375607-103811)
Skipping upstream for region 3 in Ps_papulans_renamed_contig_1: invalid coordinates (432686-434186)
Skipping downstream for region 3 in Ps_papulans_renamed_contig_1: invalid coordinates (458413-103811)
Skipping upstream for region 4 in Ps_papulans_renamed_contig_1: invalid coordinates (2678618-2680118)
Skipping downstream for region 4 in Ps_papulans_renamed_contig_1: invalid coordinates (2692497-103811)
Skipping upstream for region 5 in Ps_papulans_renamed_contig_1: invalid coordinates (2812569-2814069)
Skipping downstream for region 5 in Ps_papulans_renamed_contig_1: invalid coordinates (

In [5]:
# Step4: Save summary table
output_df = pd.DataFrame(output_rows)
output_df.to_csv('extracted_regions_summary_1500.tsv', sep='\t', index=False)